## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
def find_candle_trend (candle_no):
    if candle_no < 10:
        return 0
    
    data = pd.read_csv(filename)
    data = data.iloc[candle_no-5:candle_no+1]
#    display (data[['Open','Close','Low','High']])
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = int(r2_score(Y, y_pred)*100)
#    print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)
    
    if coeficient > 0:
        Current_Market = 1 *  Current_Market_Fit## Bullish / Buy ##
    else:
        Current_Market = -1 * Current_Market_Fit ## Bearish / Sell ##
    
    return Current_Market

In [5]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    instrument = cfg['currency']['instrument']

In [6]:
print('Instrument:', instrument)

Instrument: EUR_USD


### The Configs for Run:

In [7]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [8]:
Load_10K_Records=True
Load_20K_Records=False

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',

currency_pairs = [instrument]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [9]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [10]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [11]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [12]:
if Load_10K_Records:
    
    #### Load 10K data
        
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
    datetime_object_15K = parser.parse(second_response['candles'][0]['time'])
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    
    if Load_20K_Records:

        #### Load 15K data

        date= datetime_object_15K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        third_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(third_response['candles'][0]['time'])
        third_response= third_response['candles']
        third_response.extend(second_response)

        #### Load 20K data

        date= datetime_object_20K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        fourth_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(fourth_response['candles'][0]['time'])
        fourth_response= fourth_response['candles']
        fourth_response.extend(third_response)

        response=fourth_response
    else:
        response=second_response 
else:
    response=first_response['candles']

In [13]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [15]:
data.shape

(9658, 12)

In [16]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-12-03,18:00:00,18,337,3,Thursday,2015-12-03T18:00:00.000000000Z,11503,1.09018,1.09813,1.09018,1.09391
1,2015-12-03,22:00:00,22,337,3,Thursday,2015-12-03T22:00:00.000000000Z,5587,1.09414,1.09566,1.08992,1.09100
2,2015-12-04,02:00:00,2,338,4,Friday,2015-12-04T02:00:00.000000000Z,3762,1.09104,1.09464,1.09046,1.09390
3,2015-12-04,06:00:00,6,338,4,Friday,2015-12-04T06:00:00.000000000Z,10055,1.09388,1.09452,1.08587,1.08899
4,2015-12-04,10:00:00,10,338,4,Friday,2015-12-04T10:00:00.000000000Z,14955,1.08901,1.09507,1.08555,1.09258


In [17]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9653,2022-02-18,02:00:00,2,49,4,Friday,2022-02-18T02:00:00.000000000Z,11727,1.13682,1.13716,1.13612,1.13659
9654,2022-02-18,06:00:00,6,49,4,Friday,2022-02-18T06:00:00.000000000Z,22465,1.13660,1.13772,1.13640,1.13710
9655,2022-02-18,10:00:00,10,49,4,Friday,2022-02-18T10:00:00.000000000Z,29630,1.13710,1.13725,1.13468,1.13490
9656,2022-02-18,14:00:00,14,49,4,Friday,2022-02-18T14:00:00.000000000Z,48016,1.13491,1.13542,1.13140,1.13224
9657,2022-02-18,18:00:00,18,49,4,Friday,2022-02-18T18:00:00.000000000Z,15439,1.13224,1.13358,1.13219,1.13237


## Simple Moving Average (SMA)

In [18]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [19]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [20]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [21]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9653,2022-02-18,02:00:00,2,49,4,Friday,2022-02-18T02:00:00.000000000Z,11727,1.13682,1.13716,1.13612,1.13659,1.1363,1.1366,1.1358,0.00029,-0.00001,0.00079
9654,2022-02-18,06:00:00,6,49,4,Friday,2022-02-18T06:00:00.000000000Z,22465,1.13660,1.13772,1.13640,1.13710,1.1365,1.1366,1.1361,0.00060,0.00050,0.00100
9655,2022-02-18,10:00:00,10,49,4,Friday,2022-02-18T10:00:00.000000000Z,29630,1.13710,1.13725,1.13468,1.13490,1.1363,1.1363,1.1362,-0.00140,-0.00140,-0.00130
9656,2022-02-18,14:00:00,14,49,4,Friday,2022-02-18T14:00:00.000000000Z,48016,1.13491,1.13542,1.13140,1.13224,1.1355,1.1358,1.1361,-0.00326,-0.00356,-0.00386
9657,2022-02-18,18:00:00,18,49,4,Friday,2022-02-18T18:00:00.000000000Z,15439,1.13224,1.13358,1.13219,1.13237,1.1346,1.1354,1.1360,-0.00223,-0.00303,-0.00363


## Price Range

In [22]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [23]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [24]:
data['Trend'] = 0

In [25]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5,Trend
0,2015-12-08,22:00:00,22,342,1,Tuesday,2015-12-08T22:00:00.000000000Z,1393,1.08918,1.08997,...,0.00203,0.00009,-0.00194,1,-0.00070,0.00009,0.00124,-0.00023,0.00052,0
1,2015-12-09,02:00:00,2,343,2,Wednesday,2015-12-09T02:00:00.000000000Z,1709,1.08988,1.09210,...,0.00272,0.00134,-0.00138,1,-0.00088,0.00134,0.00050,-0.00070,-0.00023,0
2,2015-12-09,06:00:00,6,343,2,Wednesday,2015-12-09T06:00:00.000000000Z,6692,1.09079,1.09375,...,0.00311,0.00065,-0.00246,1,-0.00231,0.00065,0.00015,-0.00088,-0.00070,0
3,2015-12-09,10:00:00,10,343,2,Wednesday,2015-12-09T10:00:00.000000000Z,6608,1.09307,1.09760,...,0.00598,0.00045,-0.00553,1,-0.00408,0.00045,0.00145,-0.00231,-0.00088,0
4,2015-12-09,14:00:00,14,343,2,Wednesday,2015-12-09T14:00:00.000000000Z,13223,1.09712,1.10047,...,0.00543,0.00021,-0.00522,1,-0.00314,0.00021,0.00208,-0.00408,-0.00231,0


In [26]:
vols = data.index.tolist()
len (vols)

9639

In [27]:
%%time
for candle_no in vols:
#    print (candle_no)
    data.at[candle_no, "Trend"] = find_candle_trend(candle_no)

CPU times: user 7min 27s, sys: 9.69 s, total: 7min 36s
Wall time: 7min 36s


In [30]:
data.tail(10)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5,Trend
9629,2022-02-17,06:00:00,6,48,3,Thursday,2022-02-17T06:00:00.000000000Z,48292,1.13566,1.13863,...,0.00435,0.00249,-0.00186,1,-0.00048,0.00249,0.00138,0.00254,-0.00082,-8
9630,2022-02-17,10:00:00,10,48,3,Thursday,2022-02-17T10:00:00.000000000Z,33519,1.13614,1.13795,...,0.00278,0.00200,-0.00078,0,0.00019,0.00200,0.00097,-0.00048,0.00254,-51
9631,2022-02-17,14:00:00,14,48,3,Thursday,2022-02-17T14:00:00.000000000Z,49834,1.13594,1.13796,...,0.00289,0.00202,-0.00087,0,0.00000,0.00202,0.00087,0.00019,-0.00048,-50
9632,2022-02-17,18:00:00,18,48,3,Thursday,2022-02-17T18:00:00.000000000Z,13845,1.13593,1.13672,...,0.00103,0.00050,-0.00053,1,-0.00029,0.00050,0.00024,0.00000,0.00019,-27
9633,2022-02-17,22:00:00,22,48,3,Thursday,2022-02-17T22:00:00.000000000Z,12446,1.13625,1.13714,...,0.00137,0.00033,-0.00104,1,-0.00056,0.00033,0.00048,-0.00029,0.00000,67
9634,2022-02-18,02:00:00,2,49,4,Friday,2022-02-18T02:00:00.000000000Z,11727,1.13682,1.13716,...,0.00104,0.00057,-0.00047,0,0.00023,0.00057,0.00070,-0.00056,-0.00029,59
9635,2022-02-18,06:00:00,6,49,4,Friday,2022-02-18T06:00:00.000000000Z,22465,1.13660,1.13772,...,0.00132,0.00062,-0.00070,1,-0.00050,0.00062,0.00020,0.00023,-0.00056,86
9636,2022-02-18,10:00:00,10,49,4,Friday,2022-02-18T10:00:00.000000000Z,29630,1.13710,1.13725,...,0.00257,0.00235,-0.00022,0,0.00220,0.00235,0.00242,-0.00050,0.00023,-3
9637,2022-02-18,14:00:00,14,49,4,Friday,2022-02-18T14:00:00.000000000Z,48016,1.13491,1.13542,...,0.00402,0.00318,-0.00084,0,0.00267,0.00318,0.00351,0.00220,-0.00050,-53
9638,2022-02-18,18:00:00,18,49,4,Friday,2022-02-18T18:00:00.000000000Z,15439,1.13224,1.13358,...,0.00139,0.00121,-0.00018,1,-0.00013,0.00121,0.00005,0.00267,0.00220,-80


In [29]:
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)